In [1]:
from sklearn.datasets import load_breast_cancer
bunch = load_breast_cancer()


In [2]:
data = bunch.data
labels = bunch.target
feature_names =bunch.feature_names


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels,
                                                    test_size=0.3,
                                                    shuffle=True,
                                                    stratify=labels)

In [4]:
from sklearn.linear_model import LogisticRegression
logisticRegression = LogisticRegression(verbose=1, n_jobs=-1) #verbose=1:訓練過程列舉 ；n_jobs=-1: 讓cpu可以用上(平行處理)


In [5]:
logisticRegression = logisticRegression.fit(X_train, y_train)
accuracy = logisticRegression.score(X_test, y_test)
print("Accuracy:", accuracy)

#用2個cpu同時執行

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Accuracy: 0.9298245614035088


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.9s finished


# -------------------------------------------------------------------
## 多加class_weight=class_weight

In [15]:
#添加class_weight
import numpy as np
unique, counts = np.unique(y_train, return_counts=True)
print(unique)
class_weight = dict(zip(unique, counts)) # 模型訓練時加上這個參數
#資料的分類不是平均的 148 vs. 250

[0 1]


In [17]:
print(counts)

[148 250]


In [16]:
 dict(zip(unique, counts))

{0: 148, 1: 250}

In [7]:
from sklearn.linear_model import LogisticRegression
logisticRegression = LogisticRegression(verbose=1, n_jobs=-1,
                                        class_weight=class_weight)  #多加上去


In [8]:
logisticRegression = logisticRegression.fit(X_train, y_train)
accuracy = logisticRegression.score(X_test, y_test)
print("Accuracy:", accuracy)
#準確率稍微提升

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Accuracy: 0.935672514619883


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.3s finished


# -------------------------------------------------------------------
## 使用import RFE 實做
### 遞迴特徵刪除:
#### 將權重平方後最小的特徵去除，再基於新的特徵進行下一輪的訓練

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE           #將權重平方後最小的特徵去除，再基於新的特徵進行下一輪的訓練
from sklearn.linear_model import LogisticRegression
from sklearn import datasets


In [10]:
# 載入資料
bunch = datasets.load_breast_cancer()
X_train, X_test, Y_train, Y_test = train_test_split(bunch.data,bunch.target,
                                                    test_size= 0.3,
                                                    shuffle=True,
                                                    stratify=bunch.target)

In [11]:
# 建立模型
# logreg = LogisticRegression(C=1e5)
logreg = LogisticRegression(C=1e5, class_weight='balanced') # class_weight='balanced':讓程式自動幫我們調整

In [12]:
# 用RFE,遞迴特徵選擇
# 參數estimator裡放機器學習模型
selector = RFE(estimator=logreg, n_features_to_select=27)  #n_features_to_select=27:設定要選擇的特徵個數 建立特徵刪選器
selector = selector.fit(X_train, Y_train)

#用selector決定到底哪些特徵要留下來，用transform把不用的特徵刪除掉
X_train = selector.transform(X_train)
X_test = selector.transform(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [13]:
# 進行訓練
logreg.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=100000.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
# 進行預測
acc = logreg.score(X_test, Y_test)
print('Accuracy:', acc)  # 準確度提高

Accuracy: 0.9415204678362573
